In [1]:
#Importando Biblotecas Necessárias para a análise
import pandas as pd
import numpy as np
import libpysal as lps
from spreg import ML_Lag
from esda.moran import Moran, Moran_Local


C:\Users\thari\anaconda3\Lib\site-packages\libpysal\cg\alpha_shapes.py:38: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
C:\Users\thari\anaconda3\Lib\site-packages\libpysal\cg\alpha_shapes.py:164: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
C:\Users\thari\anaconda3\Lib\site-packages\libpysal\cg\alpha_shapes.py:198: NumbaDeprecationWarning: The 'nopyt

In [2]:
# Coordenadas das subprefeituras
coordenadas = pd.read_csv('coordenadas.csv')

# Nomes das subprefeituras
subprefeituras = [
    'Lapa', 'Pinheiros', 'Butantã', 'Itaim Paulista', 'Vila Mariana', 'Santo Amaro', 'Jabaquara', 'Sé',
    'Vila Maria/Vila Guilherme', 'Santana/Tucuruvi', 'Jaçanã/Tremembé', 'Moóca', 'Sapopemba', 'Vila Prudente',
    'Penha', 'Aricanduva/Formosa/Carrão', 'Perus', 'Freguesia do Ó/Brasilândia', 'Casa Verde/Cachoeirinha',
    'Pirituba', 'Itaquera', 'Ermelino Matarazzo', 'Guaianases', 'São Mateus', 'Ipiranga', 'São Miguel',
    'Cidade Tiradentes', 'Campo Limpo', 'Cidade Ademar', 'Capela do Socorro', 'Parelheiros', 'M\'Boi Mirim'
]

# Verificando se as listas têm o mesmo comprimento
assert len(coordenadas['Latitude']) == len(coordenadas['Longitude']) == len(subprefeituras), "As listas devem ter o mesmo comprimento."

# criando um DataFrame para as subprefeituras
df_subprefeituras = pd.DataFrame({
    'Subprefeitura': subprefeituras,
    'Latitude': coordenadas['Latitude'],
    'Longitude': coordenadas['Longitude']
})

# Lendo dados de renda correspondendo ao número de subprefeituras
data = pd.read_csv('renda.csv')
  
# Adicionando os dados de renda ao DataFrame
df_subprefeituras['Renda'] = data['Renda']

In [3]:
# Criando a matriz de pesos espaciais
coords = df_subprefeituras[['Latitude', 'Longitude']].values
k = 3
w = lps.weights.KNN(coords, k)

# Criando um modelo SAR (Spatial Autoregressive Model)
sar_model = ML_Lag(y=df_subprefeituras['Renda'].values, x=df_subprefeituras[['Latitude', 'Longitude']].values, w=w, name_x=['Latitude', 'Longitude'])

# Ajustando o modelo SAR
sar_results = sar_model

# Calcule os valores previstos
valores_previstos = sar_results.predy.flatten()

C:\Users\thari\anaconda3\Lib\site-packages\scipy\optimize\_minimize.py:892: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "


In [4]:
# Calculando o efeito de spillover (diferença entre valores reais e previstos)
efeito_spillover = df_subprefeituras['Renda'].values - valores_previstos

In [5]:
# Realizando a análise de autocorrelação espacial global
moran = Moran(efeito_spillover, w)
print("Índice de Moran Global:", moran.I)
print("Valor esperado sob aleatoriedade:", moran.EI)
print("Valor-p:", moran.p_sim)

Índice de Moran Global: 0.06774558653121264
Valor esperado sob aleatoriedade: -0.03225806451612903
Valor-p: 0.21


In [7]:
# Calculando a análise de autocorrelação espacial local
moran_local = Moran_Local(efeito_spillover, w)

# Você pode examinar os clusters espaciais significativos com base no Moran Local
significant_clusters = moran_local.p_sim < 0.05
print("Clusters Espaciais Significativos:", significant_clusters)

Clusters Espaciais Significativos: [ True False False False False  True False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False]


In [8]:
# Grau de spillover
spillover = sar_results.rho
print("Grau de Spillover:", spillover)

Grau de Spillover: 0.20711706939570818
